## Calculate annual statistic per time seres (for each pixel) within a single Equi7grid tile

In [1]:
import numpy as np
import os, osr, glob
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime
import pandas as pd
import xarray as xr
import rioxarray
import zarr
# import TUW packages
from yeoda.products.preprocessed import SIG0DataCube
from geopathfinder.naming_conventions.yeoda_naming import YeodaFilename
#
%load_ext autoreload
%autoreload 2
%reload_ext autoreload
# import my aux functions
from auxilary_ts_tools_mm import plot_TS_fromPandaSeres, features_from_S1_TS, features_as_xrrray_ufunc

In [2]:
from ignore_broken_tiffs import *

In [3]:
myTile = 'E081N069T3'
my_chunk_size = 250
my_tile_size = 15000
#
my_row = 57
my_col = 7

Specify the folder (Equi7 tile) with S1 data (a 300x300 km<sup>2</sup>)

In [4]:
tile_dir1_path = r'/project/return/Share/EODC_SA020M/V01R01/' + myTile
tile_dir2_path = r'/project/return/Share/EODC_SA020M/V1M0R1/' + myTile
# specify other parameters:
dimensions=['time', 'band', 'extra_field', 'sensor_field', 'data_version']
# get the filenames
filepaths1 = glob.glob(os.path.join(tile_dir1_path,'*.tif'))
filepaths2 = glob.glob(os.path.join(tile_dir2_path,'*.tif'))

Get the lists of all S1 images in the two eodc folders with S1 data:

In [5]:
tile_names1 = [os.path.basename(aa) for aa in glob.glob(r'/project/return/Share/EODC_SA020M/V01R01/*')]
tile_names2 = [os.path.basename(aa) for aa in glob.glob(r'/project/return/Share/EODC_SA020M/V1M0R1/*')]

In [6]:
# just a small check:
if len(tile_names1) == len(tile_names2):
    print('The number of files is identical')
else:
    print('File number not identical')

The number of files is identical


Read the datacube:

In [7]:
sig0_dc1 = SIG0DataCube(filepaths=filepaths1, dimensions=dimensions, filename_class=YeodaFilename, sres=20, continent='SA')
sig0_dc2 = SIG0DataCube(filepaths=filepaths2, dimensions=dimensions, filename_class=YeodaFilename, sres=20, continent='SA')
# get info:
sig0_dc2.inventory[dimensions].head(5)

,time,band,extra_field,sensor_field,data_version,var_name,tile_name
0,2022-05-25 09:06:24,VV,D068,S1AIWGRDH,V1M0R1,SIG0,E081N069T3
1,2018-01-10 09:05:10,VH,D068,S1AIWGRDH,V1M0R1,SIG0,E081N069T3
2,2017-09-17 09:14:08,VV,D141,S1AIWGRDH,V1M0R1,SIG0,E081N069T3
3,2018-04-04 09:05:35,VV,D068,S1AIWGRDH,V1M0R1,SIG0,E081N069T3
4,2017-04-02 09:13:40,VV,D141,S1AIWGRDH,V1M0R1,SIG0,E081N069T3


Filter by date:

In [8]:
#toi_start, toi_end = datetime(2017, 1, 1), datetime(2023, 1, 1)
toi_start, toi_end = datetime(2017, 1, 1), datetime(2021, 1, 1)
sig0_dc1 = sig0_dc1.filter_by_dimension([(toi_start, toi_end)], [(">=", "<")], name="time", inplace=True)
sig0_dc2 = sig0_dc2.filter_by_dimension([(toi_start, toi_end)], [(">=", "<")], name="time", inplace=True)

Select bands:

In [9]:
sig0_vv_dc1 = sig0_dc1.filter_by_dimension('VV', name='band')
sig0_vh_dc1 = sig0_dc1.filter_by_dimension('VH', name='band')
#
sig0_vv_dc2 = sig0_dc2.filter_by_dimension('VV', name='band')
sig0_vh_dc2 = sig0_dc2.filter_by_dimension('VH', name='band')

Merge and sort the datacubes:

In [10]:
sig0_vv_dc = sig0_vv_dc1.unite(sig0_vv_dc2)
sig0_vv_dc = sig0_vv_dc.sort_by_dimension('time', ascending=True)
#
sig0_vh_dc = sig0_vh_dc1.unite(sig0_vh_dc2)
sig0_vh_dc = sig0_vh_dc.sort_by_dimension('time', ascending=True)
#
sig0_vv_dc.inventory

,filepath,time,band,extra_field,sensor_field,data_version,var_name,tile_name
627,/project/return/Share/EODC_SA020M/V1M0R1/E081N...,2017-01-03 09:05:03,VV,D068,S1AIWGRDH,V1M0R1,SIG0,E081N069T3
417,/project/return/Share/EODC_SA020M/V1M0R1/E081N...,2017-01-03 09:05:28,VV,D068,S1AIWGRDH,V1M0R1,SIG0,E081N069T3
449,/project/return/Share/EODC_SA020M/V1M0R1/E081N...,2017-01-03 09:05:53,VV,D068,S1AIWGRDH,V1M0R1,SIG0,E081N069T3
386,/project/return/Share/EODC_SA020M/V1M0R1/E081N...,2017-01-08 09:13:04,VV,D141,S1AIWGRDH,V1M0R1,SIG0,E081N069T3
681,/project/return/Share/EODC_SA020M/V1M0R1/E081N...,2017-01-08 09:13:29,VV,D141,S1AIWGRDH,V1M0R1,SIG0,E081N069T3
...,...,...,...,...,...,...,...,...
199,/project/return/Share/EODC_SA020M/V01R01/E081N...,2020-12-25 09:05:55,VV,D068,S1A,V01R01,SIG0,E081N069T3
203,/project/return/Share/EODC_SA020M/V01R01/E081N...,2020-12-25 09:06:20,VV,D068,S1A,V01R01,SIG0,E081N069T3
52,/project/return/Share/EODC_SA020M/V01R01/E081N...,2020-12-30 09:13:36,VV,D141,S1A,V01R01,SIG0,E081N069T3
166,/project/return/Share/EODC_SA020M/V01R01/E081N...,2020-12-30 09:14:01,VV,D141,S1A,V01R01,SIG0,E081N069T3


Select only descending orbits

Specify indexing for looping trough individual chunks within the Equi7grid tile:

In [11]:
desc_list = [aa for aa in sig0_vh_dc.inventory.extra_field.unique().tolist() if aa[0]=='D']
desc_list

['D068', 'D141']

In [12]:
myOrbit = 'D068'

In [13]:
sig0_vv_orbit_dc = sig0_vv_dc.filter_by_dimension(myOrbit, name="extra_field")
sig0_vv_orbit_dc = sig0_vv_orbit_dc.sort_by_dimension('time', ascending=True)
#
sig0_vh_orbit_dc = sig0_vh_dc.filter_by_dimension(myOrbit, name="extra_field")
sig0_vh_orbit_dc = sig0_vh_orbit_dc.sort_by_dimension('time', ascending=True)
#
sig0_vv_orbit_dc.inventory.head()

,filepath,time,band,extra_field,sensor_field,data_version,var_name,tile_name
0,/project/return/Share/EODC_SA020M/V1M0R1/E081N...,2017-01-03 09:05:03,VV,D068,S1AIWGRDH,V1M0R1,SIG0,E081N069T3
1,/project/return/Share/EODC_SA020M/V1M0R1/E081N...,2017-01-03 09:05:28,VV,D068,S1AIWGRDH,V1M0R1,SIG0,E081N069T3
2,/project/return/Share/EODC_SA020M/V1M0R1/E081N...,2017-01-03 09:05:53,VV,D068,S1AIWGRDH,V1M0R1,SIG0,E081N069T3
3,/project/return/Share/EODC_SA020M/V1M0R1/E081N...,2017-01-15 09:05:03,VV,D068,S1AIWGRDH,V1M0R1,SIG0,E081N069T3
4,/project/return/Share/EODC_SA020M/V1M0R1/E081N...,2017-01-15 09:05:28,VV,D068,S1AIWGRDH,V1M0R1,SIG0,E081N069T3


In [25]:
remove_broken_files(sig0_vh_orbit_dc.inventory)

,filepath,time,band,extra_field,sensor_field,data_version,var_name,tile_name


In [27]:
[os.path.basename(aa) for aa in sig0_vh_orbit_dc.inventory.filepath]

['SIG0_20170103T090503__VH_D068_E081N069T3_SA020M_V1M0R1_S1AIWGRDH.tif',
 'SIG0_20170103T090528__VH_D068_E081N069T3_SA020M_V1M0R1_S1AIWGRDH.tif',
 'SIG0_20170103T090553__VH_D068_E081N069T3_SA020M_V1M0R1_S1AIWGRDH.tif',
 'SIG0_20170115T090503__VH_D068_E081N069T3_SA020M_V1M0R1_S1AIWGRDH.tif',
 'SIG0_20170115T090528__VH_D068_E081N069T3_SA020M_V1M0R1_S1AIWGRDH.tif',
 'SIG0_20170115T090553__VH_D068_E081N069T3_SA020M_V1M0R1_S1AIWGRDH.tif',
 'SIG0_20170127T090503__VH_D068_E081N069T3_SA020M_V1M0R1_S1AIWGRDH.tif',
 'SIG0_20170127T090528__VH_D068_E081N069T3_SA020M_V1M0R1_S1AIWGRDH.tif',
 'SIG0_20170127T090553__VH_D068_E081N069T3_SA020M_V1M0R1_S1AIWGRDH.tif',
 'SIG0_20170208T090503__VH_D068_E081N069T3_SA020M_V1M0R1_S1AIWGRDH.tif',
 'SIG0_20170208T090528__VH_D068_E081N069T3_SA020M_V1M0R1_S1AIWGRDH.tif',
 'SIG0_20170208T090553__VH_D068_E081N069T3_SA020M_V1M0R1_S1AIWGRDH.tif',
 'SIG0_20170220T090528__VH_D068_E081N069T3_SA020M_V1M0R1_S1AIWGRDH.tif',
 'SIG0_20170220T090553__VH_D068_E081N069T3_SA020M_V

Load data 

In [28]:
# read a single orbit
sig0_vv_dc_chunk1 = sig0_vv_orbit_dc.load_by_pixels((my_row-1)*my_chunk_size, (my_col-1)*my_chunk_size, row_size=250, col_size=250, dtype='xarray')
sig0_vh_dc_chunk1 = sig0_vh_orbit_dc.load_by_pixels((my_row-1)*my_chunk_size, (my_col-1)*my_chunk_size, row_size=250, col_size=250, dtype='xarray')

ERROR 1: TIFFFillStrip:Read error at scanline 4294967295; got 0 bytes, expected 30
ERROR 1: TIFFReadEncodedStrip() failed.
ERROR 1: /project/return/Share/EODC_SA020M/V1M0R1/E081N069T3/SIG0_20180110T090535__VH_D068_E081N069T3_SA020M_V1M0R1_S1AIWGRDH.tif, band 1: IReadBlock failed at X offset 0, Y offset 2800: TIFFReadEncodedStrip() failed.


LoadingDataError: Failed loading the data.

Rename the variavle

In [ ]:
sig0_vv_dc_chunk1 = sig0_vv_dc_chunk1.rename({'1':'sig0_vv'})

In [ ]:
sig0_vh_dc_chunk1 = sig0_vh_dc_chunk1.rename({'1':'sig0_vh'})

In [ ]:
# print(sig0_vh_dc_chunk1)

Rescale the data in 2019 and 2020

In [ ]:
sig0_vv_dc_chunk1['sig0_vv'].loc[slice('2019-1-1','2021-1-1'), :, :] = sig0_vv_dc_chunk1.sel(time=slice('2019-1-1','2021-1-1')).apply(lambda x: np.round(x/10.,1)).sig0_vv.values
sig0_vh_dc_chunk1['sig0_vh'].loc[slice('2019-1-1','2021-1-1'), :, :] = sig0_vh_dc_chunk1.sel(time=slice('2019-1-1','2021-1-1')).apply(lambda x: np.round(x/10.,1)).sig0_vh.values

In [ ]:
sig0_vv_dc_chunk1['sig0_vv'].isnull().values.all()

In [ ]:
sig0_vh_dc_chunk1.sig0_vh.sel(time=slice('2017-01-01', '2018-01-01')).mean('time')

In [ ]:
sig0_vh_dc_chunk1.sig0_vh.sel(time=slice('2017-01-01', '2018-01-01')).mean('time').plot(cmap='Greys_r')

In [ ]:
%matplotlib inline
#sig0_vh_dc_chunk1.sig0_vh.min(dim='time').plot(cmap='Greys_r')
sig0_vh_dc_chunk1.sig0_vh.min(dim='time').plot(cmap='Greys_r')

# Inspect Single orbit time-seres

convert to panda seres:

In [ ]:
sig0_vv_ts = sig0_vv_dc_chunk1.sig0_vv.isel(x=249, y=249).dropna(dim='time').to_series()
#sig0_vv_ts = sig0_vv_dc_chunk1.sig0_vv.sel(x=7911673, y=6710188, method="nearest").dropna(dim='time').to_series()

In [ ]:
my_xticks = pd.date_range(datetime(2017,1,1), datetime(2021,1,1), freq='YS')
sig0_vv_ts.plot(style='ro-', xticks=my_xticks, grid=True, figsize=(14,4), legend=True, xlabel='Time', ylabel='Bacscatter Intensity [dB]')

Round the time and resample to 6 day TS:

In [ ]:
sig0_vv_ts.index = sig0_vv_ts.index.round('D')
#
sig0_vv_ts_6d = sig0_vv_ts.resample('6D').interpolate(method='linear')

In [ ]:
sig0_vv_ts_6d.plot(style='bo-',grid=True, figsize=(14,4), legend=True, xlabel='Time', ylabel='Bacscatter Intensity [dB]')

In [ ]:
#%matplotlib widget
%matplotlib inline
plot_TS_fromPandaSeres(sig0_vv_ts_6d)